In [1379]:
import pandas as pd
import altair as alt
import requests
from textwrap3 import wrap
from vega_datasets import data
alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('data_server')

In [14]:
df = pd.read_csv('../data/kaggle_survey_2022_responses.csv', skiprows=[0])
df

/var/folders/60/92v1zbc11svf39kz6pl6njxh0000gn/T/ipykernel_85503/2085735099.py:1: DtypeWarning: Columns (208,225,255,257,260,270,271,277) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/kaggle_survey_2022_responses.csv', skiprows=[0])


,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,"Are you currently a student? (high school, university, or graduate)",On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Coursera,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - edX,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Kaggle Learn Courses,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - DataCamp,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Fast.ai,...,"Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Reddit (r/machinelearning, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Kaggle (notebooks, forums, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Course Forums (forums.fast.ai, Coursera forums, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - YouTube (Kaggle YouTube, Cloud AI Adventures, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Podcasts (Chai Time Data Science, O’Reilly Data Show, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Blogs (Towards Data Science, Analytics Vidhya, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Journal Publications (peer-reviewed journals, conference proceedings, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Slack Communities (ods.ai, kagglenoobs, etc)",Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - None,Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Other
0,121,30-34,Man,India,No,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,462,30-34,Man,Algeria,No,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,293,18-21,Man,Egypt,Yes,Coursera,edX,NaN,DataCamp,NaN,...,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...","Podcasts (Chai Time Data Science, O’Reilly Dat...",NaN,NaN,NaN,NaN,NaN
3,851,55-59,Man,France,No,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,...,NaN,"Kaggle (notebooks, forums, etc)","Course Forums (forums.fast.ai, Coursera forums...",NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,NaN
4,232,45-49,Man,India,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,331,22-24,Man,United States of America,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...","Podcasts (Chai Time Data Science, O’Reilly Dat...",NaN,"Journal Publications (peer-reviewed journals, ...",NaN,NaN,NaN
23993,330,60-69,Man,United States of America,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,NaN,NaN,NaN,NaN,NaN
23994,860,25-29,Man,Turkey,No,NaN,NaN,NaN,DataCamp,NaN,...,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,NaN,NaN,NaN,NaN,NaN
23995,597,35-39,Woman,Israel,No,NaN,NaN,Kaggle 

In [15]:
df = df[df['What is your gender? - Selected Choice'].notna()]
df = df.rename(columns={'What is your gender? - Selected Choice' : 'What is your gender?'})
df

,Duration (in seconds),What is your age (# years)?,What is your gender?,In which country do you currently reside?,"Are you currently a student? (high school, university, or graduate)",On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Coursera,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - edX,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Kaggle Learn Courses,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - DataCamp,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Fast.ai,...,"Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Reddit (r/machinelearning, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Kaggle (notebooks, forums, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Course Forums (forums.fast.ai, Coursera forums, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - YouTube (Kaggle YouTube, Cloud AI Adventures, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Podcasts (Chai Time Data Science, O’Reilly Data Show, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Blogs (Towards Data Science, Analytics Vidhya, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Journal Publications (peer-reviewed journals, conference proceedings, etc)","Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Slack Communities (ods.ai, kagglenoobs, etc)",Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - None,Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Other
0,121,30-34,Man,India,No,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,462,30-34,Man,Algeria,No,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,293,18-21,Man,Egypt,Yes,Coursera,edX,NaN,DataCamp,NaN,...,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...","Podcasts (Chai Time Data Science, O’Reilly Dat...",NaN,NaN,NaN,NaN,NaN
3,851,55-59,Man,France,No,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,...,NaN,"Kaggle (notebooks, forums, etc)","Course Forums (forums.fast.ai, Coursera forums...",NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,NaN
4,232,45-49,Man,India,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,331,22-24,Man,United States of America,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...","Podcasts (Chai Time Data Science, O’Reilly Dat...",NaN,"Journal Publications (peer-reviewed journals, ...",NaN,NaN,NaN
23993,330,60-69,Man,United States of America,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,NaN,NaN,NaN,NaN,NaN
23994,860,25-29,Man,Turkey,No,NaN,NaN,NaN,DataCamp,NaN,...,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,NaN,NaN,NaN,NaN,NaN
23995,597,35-39,Woman,Israel,No,NaN,NaN,Kaggle Learn Courses,NaN,

In [16]:
df['What is your age (# years)?'].value_counts()

18-21    4559
25-29    4472
22-24    4283
30-34    2972
35-39    2353
40-44    1927
45-49    1253
50-54     914
55-59     611
60-69     526
70+       127
Name: What is your age (# years)?, dtype: int64

In [413]:
gender_chart = alt.Chart(df[['What is your gender?']]).mark_bar().encode(
    alt.Y("What is your gender?", sort='-x'),
    x='count()',
    # color=alt.Color('What is your gender?', scale=alt.Scale(scheme='accent'), legend=None),
    tooltip='count()'
).properties(
    width=400,
    height=300
)
gender_chart

/Users/lzung/opt/miniconda3/envs/wids-2022/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [1045]:
gender_counts = df.groupby('What is your gender?').size().reset_index(name='counts')
gender_counts

,What is your gender?,counts
0,Man,18266
1,Nonbinary,78
2,Prefer not to say,334
3,Prefer to self-describe,33
4,Woman,5286


In [1082]:
gender_pie = alt.Chart(gender_counts).mark_arc(innerRadius=100).encode(
    theta=alt.Theta('counts', type="quantitative", stack=True),
    color=alt.Color('What is your gender?', sort=['Man', 'Woman', 'Nonbinary', 'Prefer to self-describe'], scale=alt.Scale(scheme='accent')),
    tooltip='counts'
).properties(
    height=400,
    width=400
)
gender_pie

alt.Chart(...)

In [1092]:
gender_pie.save('../img/gender_pie.html', embed_options={'renderer': 'png'})

In [23]:
age_subset = df.loc[(df['What is your gender?'] == 'Man') | (df['What is your gender?'] == 'Woman'), ['What is your age (# years)?', 'What is your gender?']]
age_subset

,What is your age (# years)?,What is your gender?
0,30-34,Man
1,30-34,Man
2,18-21,Man
3,55-59,Man
4,45-49,Man
...,...,...
23992,22-24,Man
23993,60-69,Man
23994,25-29,Man
23995,35-39,Woman


In [1334]:
age_chart = alt.Chart(age_subset).transform_joinaggregate(
    total='count(*)',
    groupby=['What is your gender?']
).transform_calculate(
    percent="1 / datum.total"
).mark_bar().encode(
    alt.X("What is your age (# years)?:N", sort=['18-21', '22-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-69', '70+'], axis=alt.Axis(labelAngle=-45)),
    alt.Y('sum(percent):Q', axis=alt.Axis(format='%'), title='Percentage of Respondents'),
    color=alt.Color('What is your gender?', scale=alt.Scale(scheme='accent'), legend=None),
    tooltip='sum(percent):Q'
).properties(
    width=300,
    height=180
).facet(
    row=alt.Row('What is your gender?', title='Gender', sort=['Man', 'Woman'])
).interactive()
age_chart

alt.FacetChart(...)

In [1335]:
age_chart.save('../img/age_chart.html', embed_options={'renderer': 'png'})

In [1336]:
demo_chart = (age_chart | gender_pie).resolve_scale(color='independent')
demo_chart

alt.HConcatChart(...)

In [1337]:
demo_chart.save('../img/demo_chart.html', embed_options={'renderer': 'png'})

In [174]:
alt.Chart(age_subset).mark_bar().encode(
    x=alt.X('What is your gender?', axis=None),
    y='count()',
    color=alt.Color('What is your gender?', scale=alt.Scale(scheme='accent')),
    column=alt.Column('What is your age (# years)?', header=alt.Header(orient='bottom')),
    tooltip='count()'
).properties(width=50).interactive()

alt.Chart(...)

In [103]:
student_subset = df.loc[(df['What is your gender?'] == 'Man') | (df['What is your gender?'] == 'Woman'), ['Are you currently a student? (high school, university, or graduate)', 'What is your gender?']]
student_subset

,"Are you currently a student? (high school, university, or graduate)",What is your gender?
0,No,Man
1,No,Man
2,Yes,Man
3,No,Man
4,Yes,Man
...,...,...
23992,Yes,Man
23993,Yes,Man
23994,No,Man
23995,No,Woman


In [1093]:
student_chart = alt.Chart(student_subset).mark_bar().encode(
    alt.X("Are you currently a student? (high school, university, or graduate)", sort=['No', 'Yes'], axis=alt.Axis(title='', labelAngle=0)),
    y='count()',
    color=alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme='accent')),
    tooltip='count()'
).properties(
    width=150,
    height=200
).facet(
    column=alt.Column('What is your gender?', sort=['Man', 'Woman'], header=alt.Header(title=['Are you currently a student?', '(high school, university, or graduate)'], labels=False))
).resolve_scale(
    x='independent'
).interactive()
student_chart

alt.FacetChart(...)

In [1338]:
student_chart = alt.Chart(student_subset).transform_joinaggregate(
    total='count(*)',
    groupby=['What is your gender?']
).transform_calculate(
    percent="1 / datum.total"
).mark_bar().encode(
    alt.X("Are you currently a student? (high school, university, or graduate)", sort=['No', 'Yes'], axis=alt.Axis(title='', labelAngle=0)),
    alt.Y('sum(percent):Q', axis=alt.Axis(format='%'), title='Percentage of Respondents'),
    color=alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme='accent')),
    column=alt.Column('What is your gender?', sort=['Man', 'Woman'], header=alt.Header(title=['Are you currently a student?', '(high school, university, or graduate)'], labels=False)),
    tooltip='sum(percent):Q'
).properties(
    width=150,
    height=200
).resolve_scale(
    y='shared'
).interactive()
student_chart

alt.Chart(...)

In [1339]:
student_chart.save('../img/student_chart.html', embed_options={'renderer': 'png'})

In [10]:
with open('questions.txt', 'w') as f:
    for q in df.columns:
        f.write(q + '\n')

In [389]:
country_df = df[['What is your gender?', 'In which country do you currently reside?']]

In [390]:
country_data = pd.read_csv("https://gist.githubusercontent.com/metal3d/5b925077e66194551df949de64e910f6/raw/c5f20a037409d96958553e2eb6b8251265c6fd63/country-coord.csv")
country_data.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


In [391]:
country_data = country_data.replace({'Country' : {'Hong Kong' : 'Hong Kong (S.A.R.)',
                                   'Iran, Islamic Republic of' : 'Iran, Islamic Republic of...',
                                   'Russian Federation' : 'Russia',
                                   'United Kingdom' : 'United Kingdom of Great Britain and Northern Ireland',
                                   'Taiwan, Province of China' : 'Taiwan',
                                   'Korea, Republic of' : 'South Korea',
                                   'United States' : 'United States of America'}})
country_data

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6
...,...,...,...,...,...,...
239,Wallis and Futuna,WF,WLF,876,-13.3000,-176.2
240,Western Sahara,EH,ESH,732,24.5000,-13.0
241,Yemen,YE,YEM,887,15.0000,48.0
242,Zambia,ZM,ZMB,894,-15.0000,30.0


In [392]:
country_df = country_df.groupby(['What is your gender?', 'In which country do you currently reside?']).size().reset_index(name='counts')
country_df.head()

,What is your gender?,In which country do you currently reside?,counts
0,Man,Algeria,44
1,Man,Argentina,145
2,Man,Australia,106
3,Man,Bangladesh,209
4,Man,Belgium,46


In [393]:
merged_country = country_df.merge(country_data, left_on='In which country do you currently reside?', right_on='Country')
merged_country

,What is your gender?,In which country do you currently reside?,counts,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Man,Algeria,44,Algeria,DZ,DZA,12,28.0,3.0
1,Prefer not to say,Algeria,2,Algeria,DZ,DZA,12,28.0,3.0
2,Woman,Algeria,16,Algeria,DZ,DZA,12,28.0,3.0
3,Man,Argentina,145,Argentina,AR,ARG,32,-34.0,-64.0
4,Prefer not to say,Argentina,2,Argentina,AR,ARG,32,-34.0,-64.0
...,...,...,...,...,...,...,...,...,...
183,Nonbinary,Viet Nam,1,Viet Nam,VN,VNM,704,16.0,106.0
184,Prefer not to say,Viet Nam,4,Viet Nam,VN,VNM,704,16.0,106.0
185,Woman,Viet Nam,28,Viet Nam,VN,VNM,704,16.0,106.0
186,Man,Zimbabwe,39,Zimbabwe,ZW,ZWE,716,-20.0,30.0


In [394]:
merged_country.sort_values('counts', ascending=False).head(10)

,What is your gender?,In which country do you currently reside?,counts,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
60,Man,India,6488,India,IN,IND,356,20.0,77.0
64,Woman,India,2179,India,IN,IND,356,20.0,77.0
177,Man,United States of America,2047,United States of America,US,USA,840,38.0,-97.0
181,Woman,United States of America,773,United States of America,US,USA,840,38.0,-97.0
17,Man,Brazil,706,Brazil,BR,BRA,76,-10.0,-55.0
105,Man,Nigeria,548,Nigeria,NG,NGA,566,10.0,8.0
108,Man,Pakistan,512,Pakistan,PK,PAK,586,30.0,70.0
81,Man,Japan,504,Japan,JP,JPN,392,36.0,138.0
30,Man,China,356,China,CN,CHN,156,35.0,105.0
92,Man,Mexico,316,Mexico,MX,MEX,484,23.0,-102.0


In [395]:
merged_country = merged_country.loc[(merged_country['What is your gender?'] == 'Man') |(merged_country['What is your gender?'] == 'Woman')]
merged_country

,What is your gender?,In which country do you currently reside?,counts,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Man,Algeria,44,Algeria,DZ,DZA,12,28.0,3.0
2,Woman,Algeria,16,Algeria,DZ,DZA,12,28.0,3.0
3,Man,Argentina,145,Argentina,AR,ARG,32,-34.0,-64.0
6,Woman,Argentina,56,Argentina,AR,ARG,32,-34.0,-64.0
7,Man,Australia,106,Australia,AU,AUS,36,-27.0,133.0
...,...,...,...,...,...,...,...,...,...
181,Woman,United States of America,773,United States of America,US,USA,840,38.0,-97.0
182,Man,Viet Nam,179,Viet Nam,VN,VNM,704,16.0,106.0
185,Woman,Viet Nam,28,Viet Nam,VN,VNM,704,16.0,106.0
186,Man,Zimbabwe,39,Zimbabwe,ZW,ZWE,716,-20.0,30.0


In [396]:
continents = pd.read_csv("https://raw.githubusercontent.com/dbouquin/IS_608/master/NanosatDB_munging/Countries-Continents.csv")
continents

,Continent,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina
...,...,...
189,South America,Paraguay
190,South America,Peru
191,South America,Suriname
192,South America,Uruguay


In [397]:
continents = continents.replace({'Country' : {'CZ' : 'Czech Republic',
                                             'Iran' : 'Iran, Islamic Republic of...',
                                             'Russian Federation' : 'Russia',
                                             'Korea, South' : 'South Korea',
                                             'United Kingdom' : 'United Kingdom of Great Britain and Northern Ireland',
                                             'US' : 'United States of America',
                                             'Vietnam' : 'Viet Nam'}})
continents = continents.append({'Country' : 'Hong Kong (S.A.R.)', 'Continent' : 'Asia'}, ignore_index=True)
continents = continents.append({'Country' : 'Taiwan', 'Continent' : 'Asia'}, ignore_index=True)
continents

/var/folders/60/92v1zbc11svf39kz6pl6njxh0000gn/T/ipykernel_85503/1944502598.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  continents = continents.append({'Country' : 'Hong Kong (S.A.R.)', 'Continent' : 'Asia'}, ignore_index=True)
/var/folders/60/92v1zbc11svf39kz6pl6njxh0000gn/T/ipykernel_85503/1944502598.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  continents = continents.append({'Country' : 'Taiwan', 'Continent' : 'Asia'}, ignore_index=True)


,Continent,Country
0,Africa,Algeria
1,Africa,Angola
2,Africa,Benin
3,Africa,Botswana
4,Africa,Burkina
...,...,...
191,South America,Suriname
192,South America,Uruguay
193,South America,Venezuela
194,Asia,Hong Kong (S.A.R.)


In [398]:
merged_country = merged_country.merge(continents, left_on='Country', right_on='Country')
merged_country

,What is your gender?,In which country do you currently reside?,counts,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average),Continent
0,Man,Algeria,44,Algeria,DZ,DZA,12,28.0,3.0,Africa
1,Woman,Algeria,16,Algeria,DZ,DZA,12,28.0,3.0,Africa
2,Man,Argentina,145,Argentina,AR,ARG,32,-34.0,-64.0,South America
3,Woman,Argentina,56,Argentina,AR,ARG,32,-34.0,-64.0,South America
4,Man,Australia,106,Australia,AU,AUS,36,-27.0,133.0,Oceania
...,...,...,...,...,...,...,...,...,...,...
107,Woman,United States of America,773,United States of America,US,USA,840,38.0,-97.0,North America
108,Man,Viet Nam,179,Viet Nam,VN,VNM,704,16.0,106.0,Asia
109,Woman,Viet Nam,28,Viet Nam,VN,VNM,704,16.0,106.0,Asia
110,Man,Zimbabwe,39,Zimbabwe,ZW,ZWE,716,-20.0,30.0,Africa


In [516]:
country_bar = alt.Chart(merged_country).transform_window(
    rank='rank(counts)',
    sort=[alt.SortField('counts', order='descending')],
    groupby=['What is your gender?']
).transform_filter(
    (alt.datum.rank <= 10)
).mark_bar().encode(
    y=alt.Y('Country', sort='-x', title=None),
    x=alt.X('counts', title='Count of Records'),
    color=alt.Color('Continent'),
    row=alt.Row('What is your gender?', header=alt.Header(labelFontSize=18, labelFontWeight='bold', title=None)),
    tooltip='counts'
).resolve_scale(y='independent').properties(width=200, height=300)
country_bar

alt.Chart(...)

In [517]:
graticule = alt.graticule()
countries = alt.topo_feature(data.world_110m.url, 'countries')

background = alt.Chart(countries).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('naturalEarth1').properties(
    width=500,
    height=300
)

men_points = (
        alt.Chart(merged_country.loc[merged_country['What is your gender?'] == 'Man'])
        .mark_circle()
        .transform_calculate(
            combined_tooltip = "datum.Country + ': ' + datum.counts"
        ).encode(
            longitude="Longitude (average):Q",
            latitude="Latitude (average):Q",
            size=alt.Size(
                "counts", scale=alt.Scale(range=(20, 1000)), title="Number of Respondents"
            ),
            color=alt.Color("Continent", title="Continent"),
            tooltip='combined_tooltip:N'
        )
    )

women_points = (
        alt.Chart(merged_country.loc[merged_country['What is your gender?'] == 'Woman'])
        .mark_circle()
        .transform_calculate(
            combined_tooltip = "datum.Country + ': ' + datum.counts"
        ).encode(
            longitude="Longitude (average):Q",
            latitude="Latitude (average):Q",
            size=alt.Size(
                "counts", scale=alt.Scale(range=(0, 1000)), title="Number of Respondents"
            ),
            color=alt.Color("Continent", title="Continent"),
            tooltip='combined_tooltip:N'
        )
    )

country_composite = country_bar | (background + men_points) & (background + women_points)
country_composite

alt.HConcatChart(...)

In [1365]:
country_df = df[['What is your gender?', 'In which country do you currently reside?']]
country_df = country_df.loc[(country_df['What is your gender?'] == 'Man') | (country_df['What is your gender?'] == 'Woman')]
country_df

,What is your gender?,In which country do you currently reside?
0,Man,India
1,Man,Algeria
2,Man,Egypt
3,Man,France
4,Man,India
...,...,...
23992,Man,United States of America
23993,Man,United States of America
23994,Man,Turkey
23995,Woman,Israel


In [1455]:
country_bar = alt.Chart(country_df).transform_joinaggregate(
    count='count(In which country do you currently reside?)',
    group_count='count(*)',
    groupby=['What is your gender?']
).transform_calculate(
    percent='1 / datum.group_count',
).mark_bar().encode(
    y=alt.Y('In which country do you currently reside?', sort='-x', title=None),
    x=alt.X('percent:Q', title='Percentage of Respondents'),
    color=alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme="accent")),
    row=alt.Row('What is your gender?', header=alt.Header(labelFontSize=0, title=None)),
    tooltip='percent:Q'
).transform_window(
    rank='rank(In which country do you currently reside?)',
    sort=[alt.SortField('percent', order='descending')],
    groupby=['What is your gender?']
).transform_filter(
    alt.datum.rank < 10
).resolve_scale(x='independent', y='independent').properties(width=200, height=300)
country_bar

alt.Chart(...)

In [1364]:
country_bar = alt.Chart(merged_country).transform_window(
    rank='rank(counts)',
    sort=[alt.SortField('counts', order='descending')],
    groupby=['What is your gender?']
).transform_filter(
    (alt.datum.rank <= 10)
).mark_bar().encode(
    y=alt.Y('Country', sort='-x', title=None),
    x=alt.X('counts', title='Number of Respondents'),
    color=alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme="accent")),
    row=alt.Row('What is your gender?', header=alt.Header(labelFontSize=0, title=None)),
    tooltip='counts'
).resolve_scale(y='independent').properties(width=200, height=300)
country_bar

alt.Chart(...)

In [1354]:
graticule = alt.graticule()
countries = alt.topo_feature(data.world_110m.url, 'countries')

background = alt.Chart(countries).mark_geoshape(
    fill='white',
    stroke='darkgrey'
).project('naturalEarth1').properties(
    width=500,
    height=300
)

men_points = (
        alt.Chart(merged_country.loc[merged_country['What is your gender?'] == 'Man'])
        .mark_circle(opacity=0.8)
        .transform_calculate(
            combined_tooltip = "datum.Country + ': ' + datum.counts"
        ).encode(
            longitude="Longitude (average):Q",
            latitude="Latitude (average):Q",
            size=alt.Size(
                "counts", scale=alt.Scale(range=(20, 1000)), title="Number of Respondents"
            ),
            color=alt.Color("What is your gender?", title='Gender', scale=alt.Scale(scheme="accent")),
            tooltip='combined_tooltip:N'
        )
    )

women_points = (
        alt.Chart(merged_country.loc[merged_country['What is your gender?'] == 'Woman'])
        .mark_circle(opacity=0.8)
        .transform_calculate(
            combined_tooltip = "datum.Country + ': ' + datum.counts"
        ).encode(
            longitude="Longitude (average):Q",
            latitude="Latitude (average):Q",
            size=alt.Size(
                "counts", scale=alt.Scale(range=(20, 1000)), title="Number of Respondents"
            ),
            color=alt.Color("What is your gender?", title='Gender', scale=alt.Scale(scheme="accent")),
            tooltip='combined_tooltip:N'
        )
    )

country_composite = country_bar | (background + men_points) & (background + women_points)
country_composite

alt.HConcatChart(...)

In [1345]:
country_composite.save('../img/country_chart.html', embed_options={'renderer': 'png'})

In [285]:
alt.Chart(countries).mark_geoshape(
    fill='lightgray',
    stroke='white'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(merged_country, 'Numeric code', list(merged_country.columns))
).mark_circle().transform_calculate(
    combined_tooltip = "datum.Country + ': ' + datum.counts"
).encode(
    longitude="Longitude (average):Q",
    latitude="Latitude (average):Q",
    size=alt.Size(
        "counts:Q", scale=alt.Scale(range=(20, 1000)), title="Number of Kaggle Users"
    ),
    color=alt.Color("Continent:N", title="Continent"),
    tooltip='combined_tooltip:Q'
).project(
    type='naturalEarth1'
).properties(
    width=600
)

alt.Chart(...)

In [662]:
years_df = df.loc[:, df.columns.str.contains('For how many years')]
years_df = years_df.join(df['What is your gender?'])
years_df['For how many years have you used machine learning methods?'] = years_df['For how many years have you used machine learning methods?'].replace(
    {'Under 1 year' : '< 1 year',
     '20 or more years' : '20+ years',
     'I do not use machine learning methods' : 'Does not use'})
years_df['For how many years have you been writing code and/or programming?'] = years_df['For how many years have you been writing code and/or programming?'].replace(
    {'< 1 years' : '< 1 year'})
years_df = years_df.dropna()
years_df

,For how many years have you been writing code and/or programming?,For how many years have you used machine learning methods?,What is your gender?
1,1-3 years,< 1 year,Man
2,1-3 years,1-2 years,Man
3,10-20 years,1-2 years,Man
4,5-10 years,Does not use,Man
5,1-3 years,1-2 years,Woman
...,...,...,...
23992,3-5 years,1-2 years,Man
23993,20+ years,2-3 years,Man
23994,< 1 year,< 1 year,Man
23995,3-5 years,< 1 year,Woman


In [663]:
years_df['For how many years have you used machine learning methods?'].unique()

array(['< 1 year', '1-2 years', 'Does not use', '4-5 years', '5-10 years',
       '3-4 years', '2-3 years', '10-20 years', '20+ years'], dtype=object)

In [664]:
years_df['For how many years have you been writing code and/or programming?'].unique()

array(['1-3 years', '10-20 years', '5-10 years', '20+ years', '3-5 years',
       '< 1 year'], dtype=object)

In [665]:
years_df = years_df.loc[(years_df['What is your gender?'] == 'Man') | (years_df['What is your gender?'] == 'Woman')]
years_df

,For how many years have you been writing code and/or programming?,For how many years have you used machine learning methods?,What is your gender?
1,1-3 years,< 1 year,Man
2,1-3 years,1-2 years,Man
3,10-20 years,1-2 years,Man
4,5-10 years,Does not use,Man
5,1-3 years,1-2 years,Woman
...,...,...,...
23992,3-5 years,1-2 years,Man
23993,20+ years,2-3 years,Man
23994,< 1 year,< 1 year,Man
23995,3-5 years,< 1 year,Woman


In [881]:
programming_chart = alt.Chart(years_df).mark_bar().encode(
    alt.X("For how many years have you been writing code and/or programming?:N",
          title=["For how many years have you been", "writing code and/or programming?"],
          sort=['< 1 year', '1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years'], axis=alt.Axis(labelAngle=-45)),
    y='count()',
    color=alt.Color('What is your gender?', scale=alt.Scale(scheme='accent'), legend=None),
    tooltip='count()'
).properties(
    width=200,
    height=200
).facet(
    row=alt.Row('What is your gender?', sort=['Man', 'Woman'])
).resolve_scale(y='shared').interactive()

ml_chart = alt.Chart(years_df).mark_bar().encode(
    alt.X("For how many years have you used machine learning methods?:N",
          title=["For how many years have you", "used machine learning methods?"],
          sort=['Does not use', '< 1 year', '1-2 years', '2-3 years', '3-4 years', '4-5 years', '5-10 years', '10-20 years', '20+ years'], axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count()', title=None),
    color=alt.Color('What is your gender?', scale=alt.Scale(scheme='accent'), title=None, legend=None),
    tooltip='count()'
).properties(
    width=300,
    height=200
).facet(
    row=alt.Row('What is your gender?', sort=['Man', 'Woman'], header=alt.Header(title=None, labelFontSize=0))
).resolve_scale(y='shared').interactive()

experience_chart = (programming_chart | ml_chart)
experience_chart

alt.HConcatChart(...)

In [1272]:
ml_dict = {'Does not use' : 0,
           '< 1 year' : 1,
           '1-2 years' : 2,
            '2-3 years' : 3,
           '3-4 years' : 4,
           '4-5 years' : 5,
           '5-10 years' : 6,
           '10-20 years' : 7,
           '20+ years' : 8}
prog_dict = {'< 1 year' : 0,
           '1-3 years' : 1,
           '3-5 years' : 2,
           '5-10 years' : 3,
           '10-20 years' : 4,
           '20+ years' : 5}

years_df['ml_code'] = years_df['For how many years have you used machine learning methods?'].map(ml_dict)
years_df['prog_code'] = years_df['For how many years have you been writing code and/or programming?'].map(prog_dict)

years_df

/var/folders/60/92v1zbc11svf39kz6pl6njxh0000gn/T/ipykernel_85503/4115807143.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  years_df['ml_code'] = years_df['For how many years have you used machine learning methods?'].map(ml_dict)
/var/folders/60/92v1zbc11svf39kz6pl6njxh0000gn/T/ipykernel_85503/4115807143.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  years_df['prog_code'] = years_df['For how many years have you been writing code and/or programming?'].map(prog_dict)


,For how many years have you been writing code and/or programming?,For how many years have you used machine learning methods?,What is your gender?,ml_code,prog_code
1,1-3 years,< 1 year,Man,1,1
2,1-3 years,1-2 years,Man,2,1
3,10-20 years,1-2 years,Man,2,4
4,5-10 years,Does not use,Man,0,3
5,1-3 years,1-2 years,Woman,2,1
...,...,...,...,...,...
23992,3-5 years,1-2 years,Man,2,2
23993,20+ years,2-3 years,Man,3,5
23994,< 1 year,< 1 year,Man,1,0
23995,3-5 years,< 1 year,Woman,1,2


In [1274]:
programming_chart_bar = alt.Chart(years_df).mark_bar().encode(
    alt.X('count()', title='Proportion of Respondents', stack="normalize"),
    alt.Y('What is your gender?', title='Gender'),
    color=alt.Color('For how many years have you been writing code and/or programming?:N',
                    title='Years of Experience',
                    scale=alt.Scale(scheme='goldorange'),
                    sort=['< 1 year', '1-3 years', '3-5 years', '5-10 years', '10-20 years', '20+ years']),
    order=alt.Order('prog_code:N', sort="ascending"),
    tooltip='count()'
).properties(
    width=500,
    height=100,
    title='For how many years have you been writing code and/or programming?'
).interactive()

ml_chart_bar = alt.Chart(years_df).mark_bar().encode(
    alt.X('count()', title='Proportion of Respondents', stack="normalize"),
    alt.Y('What is your gender?', title='Gender'),
    color=alt.Color("For how many years have you used machine learning methods?:N",
                    title='Years of Experience',
                    scale=alt.Scale(scheme='bluegreen'),
                    sort=['Does not use', '< 1 year', '1-2 years', '2-3 years', '3-4 years', '4-5 years', '5-10 years', '10-20 years', '20+ years']),
    order=alt.Order("ml_code:N", sort="ascending"),
    tooltip='count()'
).properties(
    width=500,
    height=100,
    title='For how many years have you used machine learning methods?'
).interactive()

experience_chart = (programming_chart_bar & ml_chart_bar).resolve_scale(color='independent')
experience_chart

alt.VConcatChart(...)

In [1275]:
experience_chart.save('../img/experience_chart.html', embed_options={'renderer': 'png'})

In [849]:
edu_df.loc[edu_df['What is your gender?'] == 'Woman'].value_counts()

What is your gender?  What is the highest level of formal education that you have attained or plan to attain within the next 2 years?
Woman                 Master’s degree                                                                                                    2018
                      Bachelor’s degree                                                                                                  1608
                      Doctoral degree                                                                                                     623
                      I prefer not to answer                                                                                              379
                      Some college/university study without earning a bachelor’s degree                                                   289
                      Professional doctorate                                                                                              148
              

In [952]:
edu_df = df[['What is your gender?', 'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?']]
edu_df = edu_df.loc[(edu_df['What is your gender?'] == 'Man') | (edu_df['What is your gender?'] == 'Woman')]
edu_df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'] = edu_df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].replace(
    {"No formal education past high school" : 'High school',
     "Some college/university study without earning a bachelor’s degree" : 'Some college/university'})
edu_df = edu_df[edu_df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'] != 'I prefer not to answer']
edu_df = edu_df.dropna()
edu_df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].unique()
# edu_df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'] = edu_df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].apply(wrap, args=[25])

array(['Master’s degree', 'Bachelor’s degree', 'Some college/university',
       'Doctoral degree', 'Professional doctorate', 'High school'],
      dtype=object)

In [1346]:
# edu_chart = alt.Chart(edu_df).mark_bar().encode(
#     alt.X("What is the highest level of formal education that you have attained or plan to attain within the next 2 years?:N",
#           title=['What is the highest level of formal education that', 'you have attained or plan to attain within the next 2 years?'],
#           sort=["I prefer not to answer",
#                 "High school",
#                 "Some college/university",
#                 "Bachelor’s degree",
#                 "Master’s degree",
#                 "Professional doctorate",
#                 "Doctoral degree"], axis=alt.Axis(labelAngle=-25)),
#     y='count()',
#     color=alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme='accent')),
#     tooltip='count()'
# ).properties(
#     width=400,
#     height=200
# ).facet(
#     row=alt.Row('What is your gender?', sort=['Man', 'Woman'], header=alt.Header(title=None, labelFontSize=0))
# ).resolve_scale(y='shared').interactive()
# edu_chart

edu_chart = alt.Chart(edu_df).transform_joinaggregate(
    total='count(*)',
    groupby=['What is your gender?']
).transform_calculate(
    percent="1 / datum.total"
).mark_bar().encode(
    alt.X("What is the highest level of formal education that you have attained or plan to attain within the next 2 years?:N",
          title=None,
          sort=["I prefer not to answer",
                "High school",
                "Some college/university",
                "Bachelor’s degree",
                "Master’s degree",
                "Professional doctorate",
                "Doctoral degree"], axis=alt.Axis(labelAngle=-25)),
    alt.Y('sum(percent):Q', axis=alt.Axis(format='%'), title='Percentage of Respondents'),
    color=alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme='accent')),
    row=alt.Row('What is your gender?', sort=['Man', 'Woman'], header=alt.Header(title=None, labels=False)),
    tooltip='sum(percent):Q'
).properties(
    width=350,
    height=220,
    title=['What is the highest level of formal education that', 'you have attained or plan to attain within the next 2 years?']
).resolve_scale(
    y='shared'
).interactive()
edu_chart

alt.Chart(...)

In [1021]:
edu_chart_norm = alt.Chart(edu_df).mark_bar().encode(
    alt.Y("What is the highest level of formal education that you have attained or plan to attain within the next 2 years?:N",
          sort=["I prefer not to answer",
                "High school",
                "Some college/university",
                "Bachelor’s degree",
                "Master’s degree",
                "Professional doctorate",
                "Doctoral degree"], axis=alt.Axis(title=None, labelAngle=0)),
    x=alt.X('count()', stack='normalize', title='Proportion of Respondents'),
    color=alt.Color('What is your gender?', scale=alt.Scale(scheme='accent')),
    order=alt.Order('count()', sort='ascending'),
    tooltip='count()'
).properties(
    width=400,
    height=200,
    title=['What is the highest level of formal education that', 'you have attained or plan to attain within the next 2 years?'],
).resolve_scale(y='shared').interactive()

edu_chart_norm

alt.Chart(...)

In [976]:
edu_df_counts = edu_df.groupby(['What is your gender?', 'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?']).size().reset_index(name='counts')
edu_df_counts

,What is your gender?,What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,counts
0,Man,Bachelor’s degree,5895
1,Man,Doctoral degree,1993
2,Man,High school,472
3,Man,Master’s degree,6969
4,Man,Professional doctorate,419
5,Man,Some college/university,1109
6,Woman,Bachelor’s degree,1608
7,Woman,Doctoral degree,623
8,Woman,High school,80
9,Woman,Master’s degree,2018


In [1005]:
edu_pie = (
    alt.Chart(edu_df_counts).mark_arc().encode(
    theta=alt.Theta('counts', type="quantitative"),
    color=alt.Color('What is your gender?', scale=alt.Scale(scheme='accent')),
    tooltip='counts'
).properties(
    height=150,
    width=150
).facet(
    facet=alt.Facet('What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',
                     sort=["I prefer not to answer",
                        "High school",
                        "Some college/university",
                        "Bachelor’s degree",
                        "Master’s degree",
                        "Professional doctorate",
                        "Doctoral degree"],
                   title=['What is the highest level of formal education', 'that you have attained or plan to attain within the next 2 years?']),
    columns=2
).resolve_scale(theta='independent')
)
edu_pie

alt.FacetChart(...)

In [955]:
edu_chart_bar = alt.Chart(edu_df).mark_bar().encode(
    alt.X('count()', title='Proportion of Respondents', stack="normalize"),
    alt.Y('What is your gender?', title='Gender'),
    color=alt.Color("What is the highest level of formal education that you have attained or plan to attain within the next 2 years?:N",
                    title='Education Level',
                    scale=alt.Scale(scheme='yellowgreenblue'),
                    sort=["I prefer not to answer",
                    "High school",
                    "Some college/university",
                    "Bachelor’s degree",
                    "Master’s degree",
                    "Professional doctorate",
                    "Doctoral degree"]),
    order=alt.Order("count()", sort="descending"),
    tooltip='count()'
).properties(
    width=500,
    height=100,
    title=["What is the highest level of formal education", "that you have attained or plan to attain within the next 2 years?"]
).interactive()
edu_chart_bar

alt.Chart(...)

In [929]:
research_df = df[['What is your gender?', 'Have you ever published any academic research (papers, preprints, conference proceedings, etc)?']]
research_df = research_df.loc[(research_df['What is your gender?'] == 'Man') | (edu_df['What is your gender?'] == 'Woman')]
research_df = research_df.dropna()

In [1347]:
# research_chart = alt.Chart(research_df).mark_bar().encode(
#     alt.X("Have you ever published any academic research (papers, preprints, conference proceedings, etc)?",
#           sort=['No', 'Yes'], axis=alt.Axis(title='', labelAngle=0)),
#     y='count()',
#     color=alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme='accent')),
#     tooltip='count()'
# ).properties(
#     width=150,
#     height=200
# ).facet(
#     column=alt.Column('What is your gender?', sort=['Man', 'Woman'], header=alt.Header(title=['Have you ever published any academic research', '(papers, preprints, conference proceedings, etc)?'], labels=False))
# ).resolve_scale(
#     x='independent'
# ).interactive()
# research_chart

research_chart = alt.Chart(research_df).transform_joinaggregate(
    total='count(*)',
    groupby=['What is your gender?']
).transform_calculate(
    percent="1 / datum.total"
).mark_bar().encode(
    alt.X("Have you ever published any academic research (papers, preprints, conference proceedings, etc)?",
          sort=['No', 'Yes'], axis=alt.Axis(title='', labelAngle=0)),
    alt.Y('sum(percent):Q', axis=alt.Axis(format='%'), title='Percentage of Respondents'),
    color=alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme='accent')),
    column=alt.Column('What is your gender?', sort=['Man', 'Woman'],
                      header=alt.Header(title=['Have you ever published any academic research', '(papers, preprints, conference proceedings, etc)?'], labels=False)),
    tooltip='sum(percent):Q'
).properties(
    width=150,
    height=200
).resolve_scale(
    y='shared'
).interactive()
research_chart

alt.Chart(...)

In [946]:
research_chart_bar = alt.Chart(research_df).mark_bar().encode(
    alt.X('count()', title='Proportion of Respondents', stack="normalize"),
    alt.Y('What is your gender?', title='Gender'),
    color=alt.Color("Have you ever published any academic research (papers, preprints, conference proceedings, etc)?",
                    title='Response',
                    scale=alt.Scale(scheme='accent'),
                    sort=['No', 'Yes']),
    order=alt.Order('count()', sort="descending"),
    tooltip='count()'
).properties(
    width=300,
    height=100,
    title=["Have you ever published any academic research", "(papers, preprints, conference proceedings, etc)?"]
).interactive()
research_chart_bar

alt.Chart(...)

In [1348]:
academia_chart = edu_chart | (student_chart & research_chart)
academia_chart

alt.HConcatChart(...)

In [1349]:
academia_chart.save('../img/academia_chart.html', embed_options={'renderer': 'png'})

In [1024]:
platform_df = df.loc[:, df.columns.str.contains('What products or platforms did you find to be most helpful')]
platform_df = platform_df.fillna(0)
platform_df[platform_df != 0] = 1
platform_df = platform_df.set_axis(['University courses', 'Online courses (Coursera, EdX, etc)', 'Social media platforms (Reddit, Twitter, etc)', 'Video platforms (YouTube, Twitch, etc)', 'Kaggle (notebooks, competitions, etc)', 'None / I do not study data science', 'Other'], axis=1)
platform_df = platform_df.join(df['What is your gender?'])
platform_df

,University courses,"Online courses (Coursera, EdX, etc)","Social media platforms (Reddit, Twitter, etc)","Video platforms (YouTube, Twitch, etc)","Kaggle (notebooks, competitions, etc)",None / I do not study data science,Other,What is your gender?
0,0,0,0,0,0,0,0,Man
1,1,0,0,0,1,0,0,Man
2,0,1,0,1,1,0,0,Man
3,0,1,0,0,1,0,0,Man
4,1,0,0,0,0,0,0,Man
...,...,...,...,...,...,...,...,...
23992,1,0,0,1,1,0,1,Man
23993,1,0,0,1,0,0,0,Man
23994,0,1,1,1,1,0,0,Man
23995,0,0,0,0,1,0,0,Woman


In [ ]:
Select the title most similar to your current role (or most recent title if retired): - Selected Choice
In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice
What is the size of the company where you are employed?
Approximately how many individuals are responsible for data science workloads at your place of business?
Does your current employer incorporate machine learning methods into their business?
Select any activities that make up an important part of your role at work: (Select all that apply) - Analyze and understand data to influence product or business decisions
Select any activities that make up an important part of your role at work: (Select all that apply) - Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data
Select any activities that make up an important part of your role at work: (Select all that apply) - Build prototypes to explore applying machine learning to new areas
Select any activities that make up an important part of your role at work: (Select all that apply) - Build and/or run a machine learning service that operationally improves my product or workflows
Select any activities that make up an important part of your role at work: (Select all that apply) - Experimentation and iteration to improve existing ML models
Select any activities that make up an important part of your role at work: (Select all that apply) - Do research that advances the state of the art of machine learning
Select any activities that make up an important part of your role at work: (Select all that apply) - None of these activities are an important part of my role at work
Select any activities that make up an important part of your role at work: (Select all that apply) - Other
What is your current yearly compensation (approximate $USD)?

In [1167]:
business_cols = ['Select the title most similar to your current role (or most recent title if retired): - Selected Choice',
                 'In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice',
                 'What is the size of the company where you are employed?',
                 'Approximately how many individuals are responsible for data science workloads at your place of business?',
                 'Does your current employer incorporate machine learning methods into their business?',
                 'What is your current yearly compensation (approximate $USD)?']

In [1170]:
business_df = df.loc[:, [c for c in df.columns if c in business_cols]]
business_df = business_df.join(df['What is your gender?'])
business_df = business_df.loc[(business_df['What is your gender?'] == 'Man') | (business_df['What is your gender?'] == 'Woman')]
business_df

,Select the title most similar to your current role (or most recent title if retired): - Selected Choice,In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice,What is the size of the company where you are employed?,Approximately how many individuals are responsible for data science workloads at your place of business?,Does your current employer incorporate machine learning methods into their business?,What is your current yearly compensation (approximate $USD)?,What is your gender?
0,NaN,NaN,NaN,NaN,NaN,NaN,Man
1,NaN,NaN,NaN,NaN,NaN,NaN,Man
2,NaN,NaN,NaN,NaN,NaN,NaN,Man
3,Data Scientist,Online Service/Internet-based Services,0-49 employees,1-2,"We recently started using ML methods (i.e., mo...","25,000-29,999",Man
4,NaN,NaN,NaN,NaN,NaN,NaN,Man
...,...,...,...,...,...,...,...
23992,NaN,NaN,NaN,NaN,NaN,NaN,Man
23993,NaN,NaN,NaN,NaN,NaN,NaN,Man
23994,Engineer (non-software),Energy/Mining,"10,000 or more employees",0,No (we do not use ML methods),"25,000-29,999",Man
23995,Research Scientist,Academics/Education,0-49 employees,0,"We recently started using ML methods (i.e., mo...","15,000-19,999",Woman


In [1171]:
business_df['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].unique()

array([nan, 'Data Scientist', 'Software Engineer', 'Research Scientist',
       'Other', 'Developer Advocate',
       'Data Analyst (Business, Marketing, Financial, Quantitative, etc)',
       'Data Engineer', 'Currently not employed',
       'Machine Learning/ MLops Engineer', 'Engineer (non-software)',
       'Teacher / professor', 'Statistician',
       'Manager (Program, Project, Operations, Executive-level, etc)',
       'Data Administrator', 'Data Architect'], dtype=object)

In [1211]:
role_df = business_df[['What is your gender?', 'Select the title most similar to your current role (or most recent title if retired): - Selected Choice']]
role_df = role_df.dropna()
role_df['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'] = role_df['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].replace(
    {'Data Analyst (Business, Marketing, Financial, Quantitative, etc)' : 'Data Analyst',
     'Manager (Program, Project, Operations, Executive-level, etc)' : 'Manager',
     'Machine Learning/ MLops Engineer' : 'ML/MLops Engineer'})
sorted(role_df['Select the title most similar to your current role (or most recent title if retired): - Selected Choice'].unique().tolist())

['Currently not employed',
 'Data Administrator',
 'Data Analyst',
 'Data Architect',
 'Data Engineer',
 'Data Scientist',
 'Developer Advocate',
 'Engineer (non-software)',
 'ML/MLops Engineer',
 'Manager',
 'Other',
 'Research Scientist',
 'Software Engineer',
 'Statistician',
 'Teacher / professor']

In [1212]:
role_chart = alt.Chart(role_df).mark_bar().encode(
    alt.X('count()', title='Proportion of Respondents', stack="normalize"),
    alt.Y('What is your gender?', title='Gender'),
    color=alt.Color('Select the title most similar to your current role (or most recent title if retired): - Selected Choice', title='Role',
                    scale=alt.Scale(scheme='turbo'),
                    sort=['Currently not employed']),
    order=alt.Order('count()', sort="descending"),
    tooltip='count()'
).properties(
    width=500,
    height=100,
    title='Select the title most similar to your current role (or most recent title if retired)'
).interactive()
role_chart

/Users/lzung/opt/miniconda3/envs/wids-2022/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [1231]:
role_chart = alt.Chart(role_df).mark_bar().encode(
    alt.X('count()', stack='normalize', title='Proportion of Respondents'),
    alt.Y('Select the title most similar to your current role (or most recent title if retired): - Selected Choice', title='Role',
          sort=['Currently not employed',
                'Data Administrator',
                 'Data Analyst',
                 'Data Architect',
                 'Data Engineer',
                 'Data Scientist',
                 'Developer Advocate',
                 'Engineer (non-software)',
                 'Manager',
                 'ML/MLops Engineer',
                 'Research Scientist',
                 'Software Engineer',
                 'Statistician',
                 'Teacher / professor',
                'Other']),
    alt.Order('What is your gender?', sort='descending'),
    alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme='accent')),
    tooltip='count()'
).properties(
    title='Current Role/Most Recent Title',
    width=250
).interactive()
role_chart

alt.Chart(...)

In [1224]:
industry_df = business_df[['What is your gender?', 'In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice']]
industry_df = industry_df.dropna()
industry_df['In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice'] = industry_df['In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice'].replace(
    {'Online Service/Internet-based Services' : 'Online/Internet-based Services'})
sorted(industry_df['In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice'].unique().tolist())

['Academics/Education',
 'Accounting/Finance',
 'Broadcasting/Communications',
 'Computers/Technology',
 'Energy/Mining',
 'Government/Public Service',
 'Insurance/Risk Assessment',
 'Manufacturing/Fabrication',
 'Marketing/CRM',
 'Medical/Pharmaceutical',
 'Non-profit/Service',
 'Online/Internet-based Services',
 'Other',
 'Retail/Sales',
 'Shipping/Transportation']

In [1315]:
industry_chart = alt.Chart(industry_df).mark_bar().encode(
    alt.X('count()', stack='normalize', title='Proportion of Respondents'),
    alt.Y('In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice', title='Industry',
          sort=['Academics/Education',
 'Accounting/Finance',
 'Broadcasting/Communications',
 'Computers/Technology',
 'Energy/Mining',
 'Government/Public Service',
 'Insurance/Risk Assessment',
 'Manufacturing/Fabrication',
 'Marketing/CRM',
 'Medical/Pharmaceutical',
 'Non-profit/Service',
 'Online/Internet-based Services',
 'Retail/Sales',
 'Shipping/Transportation',
               'Other']),
    alt.Order('What is your gender?', sort='descending'),
    alt.Color('What is your gender?', title='Gender', scale=alt.Scale(scheme='accent')),
    tooltip='count()'
).properties(
    title=['Industry of Current or', 'Most Recent Employer/Contract'],
    width=250
)
industry_chart

alt.Chart(...)

In [1316]:
salary_df = business_df[['What is your gender?', 'What is your current yearly compensation (approximate $USD)?']]
salary_df = salary_df.dropna()
salary_df['What is your current yearly compensation (approximate $USD)?'] = salary_df['What is your current yearly compensation (approximate $USD)?'].str.replace('$', '')
# industry_df['In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice'] = industry_df['In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice'].replace(
#     {'Online Service/Internet-based Services' : 'Online/Internet-based Services'})
sorted(salary_df['What is your current yearly compensation (approximate $USD)?'].unique().tolist())

/var/folders/60/92v1zbc11svf39kz6pl6njxh0000gn/T/ipykernel_85503/3929508954.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  salary_df['What is your current yearly compensation (approximate $USD)?'] = salary_df['What is your current yearly compensation (approximate $USD)?'].str.replace('$', '')


['0-999',
 '1,000-1,999',
 '10,000-14,999',
 '100,000-124,999',
 '125,000-149,999',
 '15,000-19,999',
 '150,000-199,999',
 '2,000-2,999',
 '20,000-24,999',
 '200,000-249,999',
 '25,000-29,999',
 '250,000-299,999',
 '3,000-3,999',
 '30,000-39,999',
 '300,000-499,999',
 '4,000-4,999',
 '40,000-49,999',
 '5,000-7,499',
 '50,000-59,999',
 '500,000-999,999',
 '60,000-69,999',
 '7,500-9,999',
 '70,000-79,999',
 '80,000-89,999',
 '90,000-99,999',
 '>1,000,000']

In [1324]:
comp_chart = alt.Chart(salary_df).transform_joinaggregate(
    total='count(*)',
    groupby=['What is your gender?']
).transform_calculate(
    percent="1 / datum.total"
).mark_bar().encode(
    alt.Y('sum(percent):Q', axis=alt.Axis(format='%'), title='Percentage of Respondents'),
    alt.X('What is your current yearly compensation (approximate $USD)?', title='Annual Salary (USD)',
          sort=['0-999',
                '1,000-1,999',
                '2,000-2,999',
                '3,000-3,999',
                '4,000-4,999',
                '5,000-7,499',
                '7,500-9,999',
                '10,000-14,999',
                '15,000-19,999',
                '20,000-24,999',
                '25,000-29,999',
                '30,000-39,999',
                '40,000-49,999',
                '50,000-59,999',
                '60,000-69,999',
                '70,000-79,999',
                '80,000-89,999',
                '90,000-99,999',
                '100,000-124,999',
                '125,000-149,999',
                '150,000-199,999',
                '200,000-249,999',
                '250,000-299,999',
                '300,000-499,999',
                '500,000-999,999',
                '>1,000,000'], axis=alt.Axis(labelAngle=-45)),
    alt.Color('What is your gender?',
              title='Gender',
              scale=alt.Scale(scheme='accent')),
    alt.Row('What is your gender?', title=None),
    tooltip='sum(percent):Q'
).properties(
    width=700,
    height=150,
    title='Current Yearly Compensation'
).resolve_scale(y='independent').interactive()
comp_chart

alt.Chart(...)

In [1350]:
business_chart = comp_chart & (role_chart | industry_chart)
business_chart

alt.VConcatChart(...)

In [1351]:
business_chart.save('../img/business_chart.html', embed_options={'renderer': 'png'})